In [1]:
import newAlgo as na
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import Competiton as cp

In [2]:
def plot_results(analysis_results):
    """Create comprehensive visualizations of the analysis results."""
    # 1. Overall Performance (Wins)
    plt.figure(figsize=(12, 6))
    plt.subplot(121)
    bars = plt.bar(analysis_results['wins'].keys(), analysis_results['wins'].values())
    plt.xlabel("Strategy")
    plt.ylabel("Wins")
    plt.title("Strategy Performance (Total Wins)")
    plt.xticks(rotation=45, ha="right")
    
    # Add value labels on top of bars
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height,
                f'{int(height)}',
                ha='center', va='bottom')
    
    # 2. Average Scores
    plt.subplot(122)
    bars = plt.bar(analysis_results['average_scores'].keys(), 
                   analysis_results['average_scores'].values())
    plt.xlabel("Strategy")
    plt.ylabel("Average Score")
    plt.title("Average Scores per Strategy")
    plt.xticks(rotation=45, ha="right")
    
    # Add value labels
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.1f}',
                ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()
    
    # 3. Payoff Matrix Heatmap
    plt.figure(figsize=(10, 8))
    plt.imshow(analysis_results['payoff_matrix'], cmap='YlOrRd')
    plt.colorbar()
    
    # Add text annotations to the heatmap
    for i in range(len(analysis_results['strategy_names'])):
        for j in range(len(analysis_results['strategy_names'])):
            plt.text(j, i, f"{analysis_results['payoff_matrix'][i, j]:.1f}",
                    ha="center", va="center")
    
    plt.xticks(range(len(analysis_results['strategy_names'])), 
               analysis_results['strategy_names'], rotation=45, ha="right")
    plt.yticks(range(len(analysis_results['strategy_names'])), 
               analysis_results['strategy_names'])
    plt.title("Average Payoff Matrix")
    plt.xlabel("Opponent Strategy")
    plt.ylabel("Strategy")
    plt.tight_layout()
    plt.show()
    
    # 4. Cooperation Rates
    plt.figure(figsize=(10, 6))
    bars = plt.bar(analysis_results['cooperation_rates'].keys(), 
                   analysis_results['cooperation_rates'].values())
    plt.xlabel("Strategy")
    plt.ylabel("Cooperation Rate")
    plt.title("Average Cooperation Rate by Strategy")
    plt.xticks(rotation=45, ha="right")
    
    # Add value labels
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.2f}',
                ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()
    
    # 5. Score Distribution Box Plot
    plt.figure(figsize=(12, 6))
    df = analysis_results['matchup_history']
    plt.boxplot([group['Score'].values for name, group in df.groupby('Strategy')],
                labels=analysis_results['strategy_names'])
    plt.xticks(rotation=45, ha="right")
    plt.ylabel("Score")
    plt.title("Score Distribution by Strategy")
    plt.tight_layout()
    plt.show()

In [ ]:
plot_results(cp.analysis_results)